In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from use_case.baseline import * 
from tests.eval import *

payoff_i = np.random.uniform(-10, 10, (20, 20))
payoff_j = np.transpose(payoff_i).copy()

# Initialize environment
N_ACTIONS = payoff_i.shape[0]
N_AGENTS = 1000
env = BaselineEnvironment(N_AGENTS, payoff_i, payoff_j, total_games = 1)

# Actual Run

In [3]:
from models.model import *
from models.trainer import *


In [4]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = 1,
    d_beliefs = 1
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [5]:
equilibriua = find_pure_equilibria(payoff_i, payoff_j)

for eq in equilibriua:
    x, y = eq 
    a = (y[0] + y[1]) / 2

    print(x, a)

(10, 10) 9.15679166251643


In [6]:
evaluate_policy(model, env, 10)

Average Return: -0.11054450682910329
Total returns: -1.1054450682910328
Action Distribution
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[  5  52  53   5  31  11  21  16  61 157  36  32  96  37  51  21 122  39
 145   9]


In [7]:
# Setup the training loop
training_parameters = TrainingParameters(
    actor_training_loops = 10,
    hypernet_training_loops= 5, 
    outer_loops = 200,
    learning_rate= 0.01,
    experience_buffer_size = 3,
)

train_model(model, env, training_parameters)
        

Epoch 0


Hypernet Loop: 100%|██████████| 5/5 [00:02<00:00,  2.10it/s]


Average Return: 2.6615315702145232
Total returns: 26.61531570214523
Action Distribution
[ 0  1  5  9 10 11 13 14 17 18 19]
[280  51  10  11   5   1  18  67 128 297 132]


Actor Training: 100%|██████████| 10/10 [00:01<00:00,  6.89it/s]


Average Return: 2.5575703596661197
Total returns: 25.575703596661196
Action Distribution
[ 0  1  5  9 10 11 13 14 17 18 19]
[254  49  18  31  13   2  67  43 147 290  86]
Epoch 1


...

Average Return: 7.333746389312582
Total returns: 73.33746389312581
Action Distribution
[ 0  1 10]
[941  58   1]
Epoch 42


Hypernet Loop: 100%|██████████| 5/5 [00:02<00:00,  2.26it/s]


Average Return: 7.459827433831498
Total returns: 74.59827433831498
Action Distribution
[ 0  1 10 11 14]
[957  40   1   1   1]


Actor Training:  20%|██        | 2/10 [00:00<00:01,  4.39it/s]


KeyboardInterrupt: 

In [ ]:
print(payoff_i[0][0])